Created by [SmirkyGraphs](http://smirkygraphs.github.io/). Code: [GitHub](https://github.com/SmirkyGraphs/Python-Notebooks). Source: [RIDOH](https://health.ri.gov/data/covid-19/).

# RI Coronavirus COVID-19 Cases

Rhode Island's Dept. of Health started releasing information regarding testing for coronavirus on their website. This information contains the following: Number of Rhode Island COVID-19 positive and presumptive positive cases, number of people who had negative test results at RIDOH's State Health Laboratory, number of pending test results, and approximate number of people who are currently instructed to self-quarantine in Rhode Island.

link to daily raw data: [here](https://github.com/SmirkyGraphs/Python-Notebooks/blob/master/ri-covid-19-tracker/data/raw/ri-covid-19.csv)<br>
link to final clean data: [here](https://github.com/SmirkyGraphs/Python-Notebooks/blob/master/ri-covid-19-tracker/data/clean/ri-covid-19-clean.csv)

In [1]:
import json
import pandas as pd
from pathlib import Path

In [2]:
# load url and name remapper
with open('./config.json') as info:
    data = json.load(info)
    url = data['url']

In [3]:
# load data from RI - DOH spreadsheet
df = pd.read_csv(url)

# transform data into table
df['date'] = list(df)[0].split(': ')[-1:][0].strip()
df.columns = ['metric', 'count', 'date']

In [4]:
# save/update file
if not Path('./data/raw/ri-covid-19.csv').exists():
    df.to_csv('./data/raw/ri-covid-19.csv', index=False)
else:
    df.to_csv('./data/raw/ri-covid-19.csv', mode='a', header=False, index=False)

In [5]:
# load raw data
df = pd.read_csv('./data/raw/ri-covid-19.csv')

# re name metrics to shorten them
df.loc[df['metric'].str.contains('positive'), 'metric'] = 'RI positive cases'
df.loc[df['metric'].str.contains('negative'), 'metric'] = 'RI negative results'
df.loc[df['metric'].str.contains('pending'), 'metric'] = 'RI pending tests'
df.loc[df['metric'].str.contains('self-quarantine'), 'metric'] = 'instructed to self-quarantine'

# clean data and get daily changes
df['new_cases'] = df.groupby('metric')['count'].diff()
df['change_%'] = df.groupby('metric')['count'].pct_change()

# export clean file
df.to_csv('./data/clean/ri-covid-19-clean.csv', index=False)